In [1]:
from __future__ import division
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
import scipy.optimize as op
import xgboost as xgb
from sklearn.metrics import mean_squared_error,make_scorer,accuracy_score,recall_score,precision_score,roc_auc_score
from sklearn.model_selection import cross_val_score, KFold, cross_validate,StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit 
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import metrics as metrices
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from statsmodels.stats.proportion import proportion_confint

%matplotlib inline

pd.options.display.float_format = '{:.3f}'.format
%config IPCompleter.greedy=True
import scipy as sc
import colorama
from colorama import Fore, Style

In [41]:
geisinger_df = pd.read_excel('../GNSIS_DATABASE_v7.5_6.25.2020.xlsx')
geisinger_afib_panda = pd.read_csv('../geisinger_afib_patients.csv',low_memory=False)
geisinger_non_afib_panda_whole = pd.read_csv('../geisinger_non_afib_patients.csv',low_memory=False)
geisinger_afib_panda.dtypes

PT_ID                                                 object
PT_SEX                                                object
PT_BIRTH_DT                                           object
PT_DEATH_DT                                           object
LAST_ACTIVE_DT                                        object
                                                      ...   
CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC           float64
DT_FOR_CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC     object
CAD_AT_INDEX                                           int64
VHD_AT_INDEX                                           int64
OBESITY_AT_INDEX                                       int64
Length: 287, dtype: object

In [24]:
Lab_data_HB = pd.read_excel('../LABS_DATASETS_CLEAN.xlsx',sheet_name='HB')
Lab_data_PLT = pd.read_excel('../LABS_DATASETS_CLEAN.xlsx',sheet_name='PLT')
Lab_data_HB_refined = Lab_data_HB[(Lab_data_HB['TIME_DIFF_HB_INDEX'] <= 0) & (Lab_data_HB['TIME_DIFF_HB_INDEX'] >- 1095)]
Lab_data_PLT_refined = Lab_data_PLT[(Lab_data_PLT['TIME_DIFF_PLT_INDEX'] <= 0) & (Lab_data_PLT['TIME_DIFF_PLT_INDEX'] > -1095)]

In [40]:
Lab_data_HB_refined = Lab_data_HB_refined.sort_values(by=['TIME_DIFF_HB_INDEX'],ascending=False).drop_duplicates(subset=['PT_ID'])
Lab_data_PLT_refined = Lab_data_PLT_refined.sort_values(by=['TIME_DIFF_PLT_INDEX'],ascending=False).drop_duplicates(subset=['PT_ID'])

In [42]:
data_afib_per_patient_panda = pd.read_csv('../df_afib_no_duplicate_per_patient.csv')


data_afib_diagnosed_before_storke = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] < 0]
data_afib_diagnosed_before_storke_at_index = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] == 0]
data_afib_diagnosed_after_storke_within_3_years =  data_afib_per_patient_panda[(data_afib_per_patient_panda['Occurance_in_years'] > 0) & (data_afib_per_patient_panda['Occurance_in_years'] <= 3)]
data_afib_diagnosed_other = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] > 3]

print(data_afib_diagnosed_before_storke['PT_ID'].nunique())
print(data_afib_diagnosed_before_storke_at_index['PT_ID'].nunique())
print(data_afib_diagnosed_after_storke_within_3_years['PT_ID'].nunique())
print(data_afib_diagnosed_other['PT_ID'].nunique())


geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke['PT_ID']), 'AFIB_DIAGNOSED'] = 1
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke_at_index['PT_ID']), 'AFIB_DIAGNOSED'] = 1
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_after_storke_within_3_years['PT_ID']), 'AFIB_DIAGNOSED'] = 1
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_other['PT_ID']), 'AFIB_DIAGNOSED'] = 0

geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke['PT_ID']), 'Group'] = 0
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke_at_index['PT_ID']), 'Group'] = 1
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_after_storke_within_3_years['PT_ID']), 'Group'] = 2
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_other['PT_ID']), 'Group'] = 3

geisinger_afib_panda['AFIB_DIAGNOSED'] = geisinger_afib_panda['AFIB_DIAGNOSED'].astype(int)
geisinger_afib_panda['Group'] = geisinger_afib_panda['Group'].astype(int)
geisinger_afib_panda['PT_ID'].nunique(),geisinger_afib_panda['Group'].value_counts()

1062
819
583
280


(2744,
 0    1062
 1     819
 2     583
 3     280
 Name: Group, dtype: int64)

In [43]:
data_full_non_afib_patients = pd.read_csv('../data_full_non_afib_patients.csv')
print(data_full_non_afib_patients['PT_ID'].nunique())
data_full_non_afib_patients['INDEX_DT'] = pd.to_datetime(data_full_non_afib_patients['INDEX_DT'])
data_full_non_afib_patients['ENC_DT'] = pd.to_datetime(data_full_non_afib_patients['ENC_DT'])
data_full_non_afib_patients['Difference'] = (data_full_non_afib_patients['ENC_DT'] - data_full_non_afib_patients['INDEX_DT'])
data_full_non_afib_patients['Difference_in_years'] = (data_full_non_afib_patients['ENC_DT'] - data_full_non_afib_patients['INDEX_DT'])/np.timedelta64(1,'Y')

data_full_non_afib_patients_no_afib_3_years_follow_up = data_full_non_afib_patients[data_full_non_afib_patients['Difference_in_years'] > 3]
print(data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID'].nunique())

data_full_non_afib_patients_no_afib_5_years_follow_up = data_full_non_afib_patients[data_full_non_afib_patients['Difference_in_years'] > 5]
print(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID'].nunique())
geisinger_non_afib_panda = geisinger_non_afib_panda_whole[geisinger_non_afib_panda_whole['PT_ID'].isin(data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID']) | geisinger_non_afib_panda['PT_ID'].isin(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID'])]
geisinger_non_afib_panda['AFIB_DIAGNOSED'] = 0
geisinger_non_afib_no_followup = geisinger_non_afib_panda_whole[~geisinger_non_afib_panda_whole['PT_ID'].isin(geisinger_non_afib_panda['PT_ID'].unique())] 


geisinger_non_afib_panda_whole.loc[geisinger_non_afib_panda_whole['PT_ID'].isin(data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID']), 'Group'] = 0
#geisinger_non_afib_panda_whole.loc[geisinger_non_afib_panda_whole['PT_ID'].isin(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID']), 'Group'] = 1
geisinger_non_afib_panda_whole.loc[geisinger_non_afib_panda_whole['PT_ID'].isin(geisinger_non_afib_no_followup['PT_ID']), 'Group'] = 1
print(data_full_non_afib_patients_no_afib_3_years_follow_up[~data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID'].isin(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID'])]['PT_ID'].nunique())
geisinger_non_afib_panda['AFIB_DIAGNOSED'].value_counts(),geisinger_non_afib_no_followup['PT_ID'].nunique(), geisinger_non_afib_panda_whole['Group'].value_counts()

6185
2190
1309
881


c:\users\mobas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(0    2190
 Name: AFIB_DIAGNOSED, dtype: int64,
 3995,
 1.000    3995
 0.000    2190
 Name: Group, dtype: int64)

In [44]:
def gender(x):
    if "Male" in x:
        return 1
    else: 
        return 0

geisinger_afib_panda["PT_SEX_CODE"] = geisinger_afib_panda["PT_SEX"].apply(gender)
geisinger_non_afib_panda_whole["PT_SEX_CODE"] = geisinger_non_afib_panda_whole["PT_SEX"].apply(gender)    

In [47]:
geisinger_non_afib_panda_whole['PT_ID'].count(),geisinger_afib_panda['PT_ID'].count()

(6185, 2744)

In [46]:
geisinger_non_afib_panda_whole = pd.merge(geisinger_non_afib_panda_whole,Lab_data_HB_refined[['PT_ID','HB_VALUE']],on=['PT_ID'],how='left')
geisinger_non_afib_panda_whole = pd.merge(geisinger_non_afib_panda_whole,Lab_data_PLT_refined[['PT_ID','PLT_VALUE']],on=['PT_ID'],how='left')

geisinger_afib_panda = pd.merge(geisinger_afib_panda,Lab_data_HB_refined[['PT_ID','HB_VALUE']],on=['PT_ID'],how='left')
geisinger_afib_panda = pd.merge(geisinger_afib_panda,Lab_data_PLT_refined[['PT_ID','PLT_VALUE']],on=['PT_ID'],how='left')

In [60]:
afib_diagnosis_pandas_limited = geisinger_afib_panda[['AGE_AT_INDEX','PT_SEX_CODE','FAM_HEART_HIST','FAM_STROKE_HIST','HYPERTENSION_AT_INDEX',
                                                               'MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX',
                                                               'HYPERCOAG_STATES_ALL_TIME','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX',
                                                               'CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX',
                                                               'CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX',
                                                               'PFO_ALL_TIME','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX',
                                                               'BMI_CLOSEST_TO_INDEX','HB_VALUE','PLT_VALUE','OBESITY_AT_INDEX','CAD_AT_INDEX','VHD_AT_INDEX','AFIB_DIAGNOSED','Group']]  

afib_diagnosis_pandas_limited_no_afib= geisinger_non_afib_panda_whole[['AGE_AT_INDEX','PT_SEX_CODE','FAM_HEART_HIST','FAM_STROKE_HIST','HYPERTENSION_AT_INDEX',
                                                               'MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX',
                                                               'HYPERCOAG_STATES_ALL_TIME','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX',
                                                               'CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX',
                                                               'CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX',
                                                               'PFO_ALL_TIME','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX',
                                                               'BMI_CLOSEST_TO_INDEX','HB_VALUE','PLT_VALUE','OBESITY_AT_INDEX','CAD_AT_INDEX','VHD_AT_INDEX','Group']]  

In [49]:
def chi_square(df,col_name, prob = 0.95, alpha = 0.05):
    flag = True
    cross= pd.crosstab(df["Group"], df[col_name])
    cross_margin= pd.crosstab(df["Group"], df[col_name],margins=True)
#     chi,p,dof,exp_freq= sc.stats.chi2_contingency(cross)
#     critical = sc.stats.chi2.ppf(prob, dof)
    print(Fore.BLUE + '\n\n------'+ col_name + '-----')
    print(Style.RESET_ALL)
    print('\n', cross_margin)
    print('\n', 100*cross_margin.iloc[:,:].div(cross_margin.All, axis=0))
    print('\n====================\n')
#     print('chi square statistics: ', chi)
#     print('critical value: ', critical)
#     print('p-value: ', p)
#     print('dof: ', dof)
#     if chi >= critical or p <= alpha :
#         print(f"{Fore.GREEN}Outcome: Significant{Style.RESET_ALL}")
#     else:
#         print(f"{Fore.RED}Outcome: Non significant{Style.RESET_ALL}")
#         flag = False
#     print('====================')
    return flag
    

In [57]:
def non_parametric_test(df,grp1,grp2,grp3,grp4,col_name, alpha = 0.05):
    flag = True
    a = df[df['Group'] == grp1][col_name].dropna()
    b = df[df['Group'] == grp2][col_name].dropna()
    c = df[df['Group'] == grp3][col_name].dropna()
    d = df[df['Group'] == grp4][col_name].dropna()
#     stat, p = sc.stats.ranksums(a, b)
    print(Fore.BLUE + '\n\n------'+ col_name + '-----')
    print(Style.RESET_ALL)
    print('grp:',grp1, 'count: ',a.count())
    print('grp:',grp1, 'std: ',a.std())
    print('grp:',grp1, 'mean: ',a.mean())
#     print('grp',grp1, 'median',a.quantile(0.5))
#     print('grp',grp1, 'IQR',a.quantile(0.75) - a.quantile(0.25))
#     print('grp',grp1, '25%',a.quantile(0.25))
#     print('grp',grp1, '75%',a.quantile(0.75))
    
    print('grp:',grp2, 'count: ',b.count())
    print('grp:',grp2, 'std: ',b.std())
    print('grp:',grp2, 'mean: ',b.mean())
#     print('grp',grp2, 'median',b.quantile(0.5))
#     print('grp',grp2, 'IQR',b.quantile(0.75) - b.quantile(0.25))
#     print('grp',grp2, '25%',b.quantile(0.25))
#     print('grp',grp2, '75%',b.quantile(0.75))
    
    print('grp:',grp3, 'count: ',c.count())
    print('grp:',grp3, 'std: ',c.std())
    print('grp:',grp3, 'mean: ',c.mean())
#     print('grp',grp3, 'median',c.quantile(0.5))
#     print('grp',grp3, 'IQR',c.quantile(0.75) - a.quantile(0.25))
#     print('grp',grp3, '25%',c.quantile(0.25))
#     print('grp',grp3, '75%',c.quantile(0.75))
    
    print('grp:',grp4, 'count: ',d.count())
    print('grp:',grp4, 'std: ',d.std())
    print('grp:',grp4, 'mean: ',d.mean())
#     print('grp',grp4, 'median',d.quantile(0.5))
#     print('grp',grp4, 'IQR',d.quantile(0.75) - a.quantile(0.25))
#     print('grp',grp4, '25%',d.quantile(0.25))
#     print('grp',grp4, '75%',d.quantile(0.75))
    print('\n')
#     print('wilcoxon statistics: ', stat)
#     print('p-value: ', p)
#     if  p <= alpha :
#         print(f"{Fore.GREEN}Outcome: Significant{Style.RESET_ALL}")
#     else:
#         print(f"{Fore.RED}Outcome: Non significant{Style.RESET_ALL}")
#         flag = False
#     print('====================')
    return flag

In [52]:
Categorical_variables_demograph=[
'PT_SEX_CODE',
'FAM_HEART_HIST',
'FAM_STROKE_HIST'
]

Categorical_variables_comorbidities=[
'HYPERTENSION_AT_INDEX',
'MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX',
'HYPERCOAG_STATES_ALL_TIME','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX',
'CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX',
'CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX',
'PFO_ALL_TIME','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX',
'OBESITY_AT_INDEX','CAD_AT_INDEX','VHD_AT_INDEX'
]

Continuous_vars=[
'AGE_AT_INDEX',
'BMI_CLOSEST_TO_INDEX',
'HB_VALUE',
'PLT_VALUE'
]

categorical_vars = Categorical_variables_demograph + Categorical_variables_comorbidities 
                 

In [53]:
# from statsmodels.imputation import mice 
# imp = mice.MICEData(afib_diagnosis_pandas_limited)
# imp.update_all(10)
# afib_diagnosis_pandas_limited = imp.data.copy()

In [54]:
sig_count = 0
non_sig_count = 0
sig_vars = []
for attr in categorical_vars:
    flag = chi_square(afib_diagnosis_pandas_limited,attr)
#     if flag:
#         sig_count+=1
#         sig_vars.append(attr)
#     else:
#         non_sig_count+=1

# print('=============Overall Stats==================')
# print(Fore.GREEN +'Significant vars count: ',sig_count)
# print(Style.RESET_ALL)
# print(Fore.RED +'Non significant vars count: ' ,non_sig_count)
# print(Style.RESET_ALL)
# print(sig_vars)



------PT_SEX_CODE-----


 PT_SEX_CODE     0     1   All
Group                        
0             556   506  1062
1             433   386   819
2             302   281   583
3             146   134   280
All          1437  1307  2744

 PT_SEX_CODE      0      1     All
Group                            
0           52.354 47.646 100.000
1           52.869 47.131 100.000
2           51.801 48.199 100.000
3           52.143 47.857 100.000
All         52.369 47.631 100.000




------FAM_HEART_HIST-----


 FAM_HEART_HIST     0     1   All
Group                           
0                592   470  1062
1                577   242   819
2                370   213   583
3                159   121   280
All             1698  1046  2744

 FAM_HEART_HIST      0      1     All
Group                               
0              55.744 44.256 100.000
1              70.452 29.548 100.000
2              63.465 36.535 100.000
3              56.786 43.214 100.000
All            61.880 38.120 100.0

In [ ]:
##### sig_count = 0
non_sig_count = 0
sig_vars=[]
for attr in Continuous_vars:
    flag = non_parametric_test(afib_diagnosis_pandas_limited,0,1,2,3,attr)
#     if flag:
#         sig_count+=1
#         sig_vars.append(attr)
#     else:
#         non_sig_count+=1

# print('=============Overall Stats==================')
# print(Fore.GREEN +'Significant vars count: ',sig_count)
# print(Style.RESET_ALL)
# print(Fore.RED +'Non significant vars count: ' ,non_sig_count)
# print(Style.RESET_ALL)
# print(sig_vars)

In [61]:
sig_count = 0
non_sig_count = 0
sig_vars = []
for attr in categorical_vars:
    flag = chi_square(afib_diagnosis_pandas_limited_no_afib,attr)
    
sig_count = 0
non_sig_count = 0
sig_vars=[]
for attr in Continuous_vars:
    flag = non_parametric_test(afib_diagnosis_pandas_limited_no_afib,0,1,2,3,attr)    



------PT_SEX_CODE-----


 PT_SEX_CODE     0     1   All
Group                        
0.0          1009  1181  2190
1.0          1873  2122  3995
All          2882  3303  6185

 PT_SEX_CODE      0      1     All
Group                            
0.0         46.073 53.927 100.000
1.0         46.884 53.116 100.000
All         46.597 53.403 100.000




------FAM_HEART_HIST-----


 FAM_HEART_HIST     0     1   All
Group                           
0.0             1227   963  2190
1.0             2836  1159  3995
All             4063  2122  6185

 FAM_HEART_HIST      0      1     All
Group                               
0.0            56.027 43.973 100.000
1.0            70.989 29.011 100.000
All            65.691 34.309 100.000




------FAM_STROKE_HIST-----


 FAM_STROKE_HIST     0    1   All
Group                           
0.0              1836  354  2190
1.0              3540  455  3995
All              5376  809  6185

 FAM_STROKE_HIST      0      1     All
Group                     